In [11]:
#http://www.jeannicholashould.com/python-web-scraping-tutorial-for-craft-beers.html
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import re

In [12]:
#regex disypher: ^ - match any character not in the set, \d - digit, $-end
# Return the beer entry numerical identifier from the "Entry" column.
def get_beer_id(cell_value):
    r = re.match("^(\d{1,4})\.$", cell_value)
    if r and len(r.groups()) == 1:
        beer_id = r.group(1)
        return int(beer_id)
    else:
        return None
    

In [13]:
#determines if row entry is a beer entry (by counting number of entries)
def is_beer_entry(table_row):
    row_cells = table_row.findAll("td")
    beer_id = get_beer_id(row_cells[0].text)
    return(len(row_cells)==8 and beer_id)

In [14]:
def get_all_beers(html_soup):
    beers = []
    all_rows_in_html_page = html_soup.findAll("tr")
    for table_row in all_rows_in_html_page:
        if is_beer_entry(table_row):
            row_cells = table_row.findAll("td")
            beer_entry = {
                "id": get_beer_id(row_cells[0].text),
                "name": row_cells[1].text,
                "brewery_name": row_cells[2].text,
                "brewery_location": row_cells[3].text,
                "style": row_cells[4].text,
                "size": row_cells[5].text,
                "abv": row_cells[6].text,    
                "ibu": row_cells[7].text
            }
            beers.append(beer_entry)
    return beers    

In [15]:
#request the page
html = urlopen("http://craftcans.com/db.php?search=all&sort=beerid&ord=desc&view=text")
html_soup = BeautifulSoup(html, 'html.parser')
beers_list = get_all_beers(html_soup)

In [16]:
df = pd.DataFrame(beers_list)
df.head(5)

,abv,brewery_location,brewery_name,ibu,id,name,size,style
0,4.5%,"Minneapolis, MN",NorthGate Brewing,50,2692,Get Together,16 oz.,American IPA
1,4.9%,"Minneapolis, MN",NorthGate Brewing,26,2691,Maggie's Leap,16 oz.,Milk / Sweet Stout
2,4.8%,"Minneapolis, MN",NorthGate Brewing,19,2690,Wall's End,16 oz.,English Brown Ale
3,6.0%,"Minneapolis, MN",NorthGate Brewing,38,2689,Pumpion,16 oz.,Pumpkin Ale
4,6.0%,"Minneapolis, MN",NorthGate Brewing,25,2688,Stronghold,16 oz.,American Porter


In [22]:
#create a sepparate breweries data to create tidy data
breweries = df[['brewery_location', 'brewery_name']]
breweries = breweries.drop_duplicates()
breweries = breweries.reset_index(drop=True)
breweries["id"] = breweries.index
breweries.head(5)

,brewery_location,brewery_name,id
0,"Minneapolis, MN",NorthGate Brewing,0
1,"Louisville, KY",Against the Grain Brewery,1
2,"Framingham, MA",Jack's Abby Craft Lagers,2
3,"San Diego, CA",Mike Hess Brewing Company,3
4,"San Francisco, CA",Fort Point Beer Company,4


In [27]:
beers = pd.merge(df,
                 breweries,
                 left_on=['brewery_name','brewery_location'],
                 right_on=['brewery_name','brewery_location'],
                 sort = True, 
                 suffixes =('_beer', '_brewery'))

beers = beers[["abv", "ibu", "id_beer",
               "name", "size", "style", "id_brewery"]]
beers_column_rename = {
    'id_beer': 'id',
    'id_brewery': 'brewery_id'
}
beers.rename(inplace=True, columns = beers_column_rename)
beers.head(5)

,abv,ibu,id,name,size,style,brewery_id
0,5.0%,N/A,1436,Pub Beer,12 oz.,American Pale Lager,408
1,6.6%,N/A,2265,Devil's Cup,12 oz.,American Pale Ale (APA),177
2,7.1%,N/A,2264,Rise of the Phoenix,12 oz.,American IPA,177
3,9.0%,N/A,2263,Sinister,12 oz.,American Double / Imperial IPA,177
4,7.5%,N/A,2262,Sex and Candy,12 oz.,American IPA,177
